# Introduction

**The &mu;-SEMNAN Solver** finds the fittest parameters in a linear Gaussian Acyclic Margincal Ancestral Structural Equation Model (AMASEM).
As we will see, there are more than one method to compute the fittest parameters of the
AMASEM structure.

We start by loading the libraries. The SEMNAN Solver uses PyTorch&reg; and depends on CUDA&reg; as the backend.
We will need to load pytorch and make sure that the backend used is CUDA. We do this by
introducing a `device` variable that is always set to cuda and pass it to any tensor we make.
We also import our library `semnan_cuda`.

In [1]:
import torch
import semnan_cuda as sc

device = torch.device("cuda")

Consider the following AMASEM.

![alt text](img/health-graph.svg "Health Graph")

It is composed of four visible variables and six latent variables.
We will compile this graph as an adgacency matrix. The encoding is simple: the latent variables go on top
and the remaining variables (the visible ones) form an upper-triangular matrix at the bottom.

In [2]:
struct = torch.tensor([
        [1, 1, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],  # V_X
        [0, 0, 1, 0],  # V_BP
        [0, 0, 0, 1],  # V_BMI
        [0, 0, 0, 0],  # V_Y
    ], dtype=torch.bool, device=device)

In order to obtain the fittest parameters, we fit this structure to a `SEMNANSolver`.


In [3]:
semnan = sc.SEMNANSolver(struct)

We will obtain the fittest parameters (that is the optimal weights between the variables)
with respect to the sample covariance matrix. It is the observed covariance matrix that has been induced by the causal system.

In [4]:
sample_covariance = torch.tensor([
        [2,  3,  6,  8],
        [3,  7, 12, 16],
        [6, 12, 23, 30],
        [8, 16, 30, 41],
    ], device=device)

semnan.sample_covariance = sample_covariance

The newly created `SEMNANSolver` object will use the gradient descent method to compute the optimal weights.
However, it only computes the partial derivatives of the objective function with respect to the weights.
Therefore, we need to use an arbitrary optimizer to update the weights in each step.

In [5]:
optim = torch.optim.Adamax([semnan.weights], lr=0.001)

That's it! We only need to start training the SEMNANSolver. Training the SEMNANSolver is pretty much
like training a neural network: we take a `forward()` and `backward()` step and then call the `step()` method
of the optimizer to do the rest. For this, we first set the stopping conditions:

In [6]:
max_iterations = 10000
min_error = 1.0e-7

The following trains the AMASEM. We would also like to print valuable information at each step of the optimizer.


In [7]:
for i in range(max_iterations):
    semnan.forward()
    error = semnan.loss().item()

    if error < min_error:
        break

    semnan.backward()
    optim.step()

    if i % (max_iterations / 10) == 0:
        print(f"iteration={i:<10} loss={error:<15.5}")
else:
    print("Did not converge in the maximum number of iterations!")

iteration=0          loss=19.145         
iteration=1000       loss=3.7843         
iteration=2000       loss=1.4576         
iteration=3000       loss=0.70791        
iteration=4000       loss=0.022899       


Now that the AMASEM has been parametrized using the `SEMNANSolver`, we can print out the induced visible covariance matrix...

In [8]:
print(semnan.visible_covariance_)

tensor([[ 1.9979,  2.9957,  5.9919,  7.9894],
        [ 2.9957,  6.9907, 11.9830, 15.9777],
        [ 5.9919, 11.9830, 22.9688, 29.9591],
        [ 7.9894, 15.9777, 29.9591, 40.9462]], device='cuda:0')


... and the weights matrix of the AMASEM.

In [9]:
print(semnan.weights)

tensor([[ 1.1341,  2.1153,  2.1113, -0.0000],
        [ 0.0000, -0.6406,  0.0000, -0.9432],
        [ 0.8437, -0.0000,  0.0000, -0.0000],
        [-0.0000, -0.7033,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.9083, -0.0000],
        [-0.0000,  0.0000,  0.0000, -0.9662],
        [ 0.0000,  0.2987,  0.9561,  0.6042],
        [ 0.0000,  0.0000,  0.5633,  0.0000],
        [ 0.0000, -0.0000,  0.0000,  1.1319],
        [ 0.0000, -0.0000, -0.0000, -0.0000]], device='cuda:0')
